In [18]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time
import sys
from sklearn.svm import SVC
import numpy as np
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, precision_score, f1_score, recall_score, roc_auc_score
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

In [19]:
# Load the data
data = pd.read_csv('train/train_radiology.csv')
labels = pd.read_csv('train/train_labels.csv')

In [20]:
# Group radiology reports by patient_id and concatenate the text
data = data.groupby('patient_id')['text'].apply(lambda x: ' '.join(x)).reset_index()

In [21]:
# Merge radiology data with label data based on patient_id
data = pd.merge(data, labels, on='patient_id', how='left')

In [22]:
# Tokenization
data['tokenized_text'] = data['text'].apply(lambda x: word_tokenize(x.lower()))

In [23]:
# Data Preprocessing
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

In [24]:
def preprocess_text(tokens):
    tokens = [token for token in tokens if token.isalpha()]  # Remove punctuation
    tokens = [token for token in tokens if token not in stop_words]  # Remove stop words
    tokens = [ps.stem(token) for token in tokens]  # Stemming
    return tokens

In [25]:
data['tokenized_text'] = data['tokenized_text'].apply(preprocess_text)

In [26]:
# Feature Extraction
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['tokenized_text'].apply(lambda x: ' '.join(x)))

In [35]:
Y = data['label']

In [28]:
# Implement Classification Algorithms
classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    MultinomialNB(),
    SVC(probability=True)
]

# Define K-Values
kVals = [0,128,256,512]

In [39]:
# Split the data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate on validation set
val_predictions = model.predict_proba(X_val)

In [40]:
val_predictions

array([[0.99, 0.01],
       [0.93, 0.07],
       [0.89, 0.11],
       ...,
       [0.83, 0.17],
       [0.96, 0.04],
       [0.89, 0.11]])

In [44]:
val_predictions = (val_predictions[:, 1] > 0.5).astype(int)

In [46]:
val_accuracy = accuracy_score(y_val, val_predictions)
print("Validation Accuracy:", val_accuracy)

# Get feature importance
feature_importance = model.feature_importances_

# Map feature names to their importance scores
feature_names = vectorizer.get_feature_names_out()
importance_dict = dict(zip(feature_names, feature_importance))

# Sort feature importance dictionary by importance score (descending order)
sorted_importance = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print top features
top_features = 10  # Number of top features to display
print(f"Top {top_features} features:")
for feature, importance in sorted_importance[:top_features]:
    print(f"{feature}: {importance}")


Validation Accuracy: 0.8910665451230629
Top 10 features:
arrest: 0.005355613391247452
right: 0.004383289199753818
left: 0.003473630690179363
chest: 0.0033639454499464995
tube: 0.003351025698247861
bilater: 0.003015778474611076
comparison: 0.002979845781214283
impress: 0.0029581257474392995
interv: 0.0029085775330486303
unchang: 0.0028790209301226555
